In [ ]:
!pip install git+https://github.com/geraldioeres/detectree2.git
!git clone https://github.com/facebookresearch/detectron2.git detectron2
!pip install rasterstats
!pip install --upgrade --force-reinstall rasterio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from detectree2.preprocessing.tiling import tile_data
from detectree2.models.outputs import project_to_geojson, stitch_crowns, clean_crowns
from detectree2.models.predict import predict_on_data
from detectree2.models.train import setup_cfg
from detectron2.engine import DefaultPredictor
import rasterio

# With Gdrive

In [ ]:
site_path = "/content/drive/MyDrive/Research/data/LS_Area"

In [ ]:
img_path = site_path + "/rgb/ls_plot_0.tif"
tiles_path = site_path + "/tilespred_temp/"

In [ ]:
def delineate_forest():
    buffer = 20
    tile_width = 30
    tile_height = 30
    tile_data(img_path, tiles_path, buffer, tile_width, tile_height, dtype_bool = True)

    resnet_model = "/content/drive/MyDrive/Research/model_resnet_pointrend/model_9.pth"
    base_model= "/content/detectron2/projects/PointRend/configs/InstanceSegmentation/pointrend_rcnn_R_101_FPN_3x_coco.yaml"

    cfg = setup_cfg(model_source="point_rend", base_model=base_model, update_model=resnet_model)
    predict_on_data(tiles_path, predictor=DefaultPredictor(cfg))
    project_to_geojson(tiles_path, tiles_path + "predictions/", tiles_path + "predictions_geo/")
    crowns = stitch_crowns(tiles_path + "predictions_geo/", 1)
    clean = clean_crowns(crowns, 0.4, confidence=0.15)
    clean.to_file(site_path + "/" + "crown_pred_" + ".gpkg")

  0%|          | 0/4 [00:00<?, ?it/s]

Predicting 4 files in mode rgb
Projecting 4 files
clearn_crowns: Performing spatial join...


clean_crowns: Processing candidate pairs: 100%|██████████| 62/62 [00:00<00:00, 907.74it/s]


# No Drive

In [ ]:
def delineate_forest(ortho_path, tiles_path):
    buffer = 20
    tile_width = 30
    tile_height = 30
    print("Tiling orthomosaic...")
    tile_data(ortho_path, tiles_path, buffer, tile_width, tile_height, dtype_bool = True)
    print("Done tiling.")

    resnet_pr_model = "/content/drive/MyDrive/Research/model_resnet_pointrend/model_9.pth"
    base_resnetpr_model= "/content/detectron2/projects/PointRend/configs/InstanceSegmentation/pointrend_rcnn_R_101_FPN_3x_coco.yaml"

    print("Delineating trees...")
    cfg = setup_cfg(model_source="point_rend", base_model=base_resnetpr_model, update_model=resnet_pr_model)
    predict_on_data(tiles_path, predictor=DefaultPredictor(cfg))
    project_to_geojson(tiles_path, tiles_path + "predictions/", tiles_path + "predictions_geo/")
    crowns = stitch_crowns(tiles_path + "predictions_geo/", 1)
    clean = clean_crowns(crowns, 0.4, confidence=0.15)
    clean.to_file("crown_pred2_" + ".gpkg")
    print("\nProcess completed.")

In [ ]:
ortho_path = "/content/drive/MyDrive/Research/data/LS_Area/rgb/ls_plot_0.tif"
tiles_path = "/tilespred_temp/"

In [ ]:
delineate_forest(ortho_path, tiles_path)

Tiling orthomosaic...


  0%|          | 0/4 [00:00<?, ?it/s]

Done tiling.
Delineating trees...
Predicting 4 files in mode rgb
Projecting 4 files
clearn_crowns: Performing spatial join...


clean_crowns: Processing candidate pairs: 100%|██████████| 62/62 [00:00<00:00, 1032.86it/s]


Process completed.


In [ ]:
buffer = 20
tile_width = 30
tile_height = 30
tile_data(ortho_path, tiles_path, buffer, tile_width, tile_height, dtype_bool = True)

  0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
resnet_model = "/content/drive/MyDrive/Research/model_resnet_pointrend/model_9.pth"
base_model= "/content/detectron2/projects/PointRend/configs/InstanceSegmentation/pointrend_rcnn_R_101_FPN_3x_coco.yaml"

In [ ]:
cfg = setup_cfg(model_source="point_rend", base_model=base_model, update_model=resnet_model)
predict_on_data(tiles_path, predictor=DefaultPredictor(cfg))
project_to_geojson(tiles_path, tiles_path + "predictions/", tiles_path + "predictions_geo/")
crowns = stitch_crowns(tiles_path + "predictions_geo/", 1)
clean = clean_crowns(crowns, 0.4, confidence=0.15)
clean.to_file("crown_pred_" + ".gpkg")

Predicting 4 files in mode rgb
Projecting 4 files
clearn_crowns: Performing spatial join...


clean_crowns: Processing candidate pairs: 100%|██████████| 62/62 [00:00<00:00, 810.81it/s]


In [ ]:
clean.head(10)

,Confidence_score,geometry
0,0.986591,"POLYGON ((221103.567 81895.254, 221103.522 818..."
1,0.976867,"POLYGON ((221086.379 81910.995, 221086.334 819..."
2,0.860601,"POLYGON ((221122.518 81869.745, 221122.473 818..."
3,0.685639,"POLYGON ((221078.781 81886.344, 221078.736 818..."
4,0.581683,"POLYGON ((221084.389 81878.474, 221084.344 818..."
5,0.554030,"POLYGON ((221109.854 81866.443, 221109.809 818..."
6,0.520195,"POLYGON ((221079.233 81871.69, 221079.188 8187..."
7,0.990094,"POLYGON ((221122.473 81856.13, 221122.428 8185..."
8,0.882420,"POLYGON ((221107.728 81843.33, 221107.683 8184..."
9,0.856325,"POLYGON ((221130.886 81840.345, 221130.841 818..."


# Final Code

In [ ]:
from detectree2.preprocessing.tiling import tile_data
from detectree2.models.outputs import project_to_geojson, stitch_crowns, clean_crowns
from detectree2.models.predict import predict_on_data
from detectree2.models.train import setup_cfg
from detectron2.engine import DefaultPredictor
import rasterio

In [ ]:
import geopandas as gpd
import math
from rasterstats import zonal_stats
import rasterio as rio
import numpy as np

In [ ]:
def delineate_forest(ortho_path, tiles_path=""):
    """
    Delineate trees on the forest's orthomosaic

    Args:
        ortho_path (str): orthomosaic file path
        tiles_path (str): tiling_path (optional)

    Returns:
        cleaned_crown (geoDataFrame): cleaned overlapping delineated tree crowns
    """

    buffer = 20
    tile_width = 30
    tile_height = 30
    print("Tiling orthomosaic...")
    if tiles_path == "":
        tiles_path = "/tilespred_temp/"
    else:
        tiles_path = tiles_path
    tile_data(ortho_path, tiles_path, buffer, tile_width, tile_height, dtype_bool = True)
    print("Done tiling.")

    resnet_pr_model = "/content/drive/MyDrive/Research/model_resnet_pointrend/model_9.pth"
    base_resnetpr_model= "/content/detectron2/projects/PointRend/configs/InstanceSegmentation/pointrend_rcnn_R_101_FPN_3x_coco.yaml"

    print("Delineating trees...")
    cfg = setup_cfg(model_source="point_rend", base_model=base_resnetpr_model, update_model=resnet_pr_model)
    predict_on_data(tiles_path, predictor=DefaultPredictor(cfg))
    project_to_geojson(tiles_path, tiles_path + "predictions/", tiles_path + "predictions_geo/")
    crowns = stitch_crowns(tiles_path + "predictions_geo/", 1)
    cleaned_crown = clean_crowns(crowns, 0.4, confidence=0.15)
    cleaned_crown.to_file("crown_pred" + ".gpkg")
    print("\nProcess completed.")

    return cleaned_crown

In [ ]:
def crown_zonalstats(crown_path, chm_path, clean=True):
    """
    Perform zonal statistic on crown and the input CHM

    Args:
        crown_path (str or geoDataFrame): crown file path or crown geoDataFrame
        chm_path (str): path of the CHM file

    Returns:
        mod_in_crown (geoDataFrame): modified crown geoDataFrame
    """

    if type(crown_path) == str:
        in_crown = gpd.read_file(crown_path)
    else:
        in_crown = crown_path
    mod_in_crown = in_crown.copy()
    crown_list = list(mod_in_crown.geometry)
    mod_in_crown["height"] = 0.00
    mod_in_crown["th_median"] = 0.00
    mod_in_crown["th_std"] = 0.00
    mod_in_crown["category"] = ""

    for c_id, crown_poly in enumerate(crown_list):
        zstat_data = zonal_stats(crown_poly, chm_path, stats="max median std")
        tree_height = zstat_data[0]["max"]
        th_median = zstat_data[0]["median"]
        th_std = zstat_data[0]["std"]
        mod_in_crown.loc[c_id, "height"] = tree_height
        mod_in_crown.loc[c_id, "th_median"] = th_median
        mod_in_crown.loc[c_id, "th_std"] = th_std

        if height <= 0:
            mod_in_crown.loc[c_id, "category"] = "Not Tree"
        else if height > 0 and (th_median < 2 or th_std < 2):
            mod_in_crown.loc[c_id, "category"] = "Not Tree"
        else:
            mod_in_crown.loc[c_id, "category"] = "Tree"

    if clean:
        mod_in_crown = mod_in_crown[mod_in_crown["category"] == "Not Tree"]

    mod_in_crown.to_file("zonal_cleaned" + ".gpkg")

    print("Crown has ben cleaned.")

    return mod_in_crown

In [ ]:
def process_chm(dsm_path, dtm_path):
    """
    Generate CHM from DSM and DTM

    Args:
        dsm_path (str): path of the DSM file
        dtm_path (str): path of the DTM file

    Returns:
        ds_chm (array): CHM in np.ndarray format
        src_dsm.meta (dict): DSM metadata
    """

    src_dsm = rio.open(dsm_path)
    src_dtm = rio.open(dtm_path)

    ds_dsm = src_dsm.read()
    ds_dtm = src_dtm.read()

    ds_dsm_c = ds_dsm.copy()
    ds_dtm_c = ds_dtm.copy()

    ds_chm = ds_dsm_c - ds_dtm_c

    return ds_chm, src_dsm.meta

In [ ]:
from rasterio.io import MemoryFile

def zonal_cleaning(dsm_path, dtm_path, crown_path, to_file=False, file_name="temp_name", clean_crown=True):
    """
    Zonal cleaning handler and CHM generation

    Args:
        dsm_path (str): path of the DSM file
        dtm_path (str): path of the DTM file
        crown_path (str or geoDataFrame): crown file path or crown geoDataFrame
        to_file(boolean): output CHM to a file
        clean_crown(boolean): clean crown

    Returns:
        mod_in_crown (geoDataFrame): modified crown geoDataFrame from
                                     crown_zonalstats() function
    """

    # CHM generation from DSM and DTM
    ds_chm, out_meta = process_chm(dsm_path, dtm_path)

    print("Performing zonal cleaning...")
    # If user want CHM output
    if to_file:
        out_meta = out_meta

        # chm_path = site_path + "/CHM_4" + ".tif"
        chm_path = file_name + ".tif"
        with rio.open(chm_path, "w", **out_meta) as dest:
            dest.write(ds_chm)

        return crown_zonalstats(crown_path, chm_path, clean_crown)

    # Otherwise, chm only as temp file
    else:
        with MemoryFile() as temp_file:
          with temp_file.open(**out_meta) as dataset:
              dataset.write(ds_chm)

          return crown_zonalstats(crown_path, temp_file, clean_crown)

In [ ]:
ortho_path = "/content/drive/MyDrive/Research/data/LS_Area/rgb/ls_plot_0.tif"

In [ ]:
site_path = "/content/drive/MyDrive/Research/data/DSTM"
dsm_path = site_path + "/DSM_Clip_3.tif"
dtm_path = site_path + "/DTM_Clip_3.tif"

In [ ]:
cleaned = delineate_forest(ortho_path)

In [ ]:
zonal_cleaned = zonal_cleaning(dsm_path, dtm_path, cleaned, to_file=True, file_name="CHM_3", clean_crown=False)

Performing zonal cleaning...
Crown has ben cleaned.


# Constants

In [ ]:
resnet_model = "/content/drive/MyDrive/Research/model_resnet/model_17.pth"

In [ ]:
resnext_model = "/content/drive/MyDrive/Research/model_resnext/model_17.pth"
base_resnext_model = "COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"

In [ ]:
resnet_pr_model = "/content/drive/MyDrive/Research/model_resnet_pointrend/model_9.pth"
base_resnetpr_model= "/content/detectron2/projects/PointRend/configs/InstanceSegmentation/pointrend_rcnn_R_101_FPN_3x_coco.yaml"

In [ ]:
resnext_pr_model = "/content/drive/MyDrive/Research/model_resnext_pointrend/model_12.pth"
base_resnextpr_model = "/content/detectron2/projects/PointRend/configs/InstanceSegmentation/pointrend_rcnn_X_101_32x8d_FPN_3x_coco.yaml"